# Messaging


### Imports

In [1]:
from aries_cloudcontroller import AriesAgentController
import os
from termcolor import colored
import asyncio
import nest_asyncio
nest_asyncio.apply()

### Initialise the Agent Controller

In [2]:
api_key = os.getenv("ACAPY_ADMIN_API_KEY")
admin_url = os.getenv("ADMIN_URL")

print(f"Initialising a controller with admin api at {admin_url} and an api key of {api_key}")
agent_controller = AriesAgentController(admin_url,api_key)

Initialising a controller with admin api at http://demo-participant-agent:3021 and an api key of demo-participantsSecretApiKey


### Start a Webhook Server

In [3]:
webhook_port = int(os.getenv("WEBHOOK_PORT"))
webhook_host = "0.0.0.0"

await agent_controller.init_webhook_server(webhook_host, webhook_port)

print(f"Listening for webhooks from agent at http://{webhook_host}:{webhook_port}")

Listening for webhooks from agent at http://0.0.0.0:3010


In [4]:
class Contact:
    
    def __init__(self, connection_id, alias, agent_label = None):
        self.connection_id = connection_id
        self.alias = alias
        self.agent_label = None
        self.inbox = []
        self.is_active = asyncio.Future()
        
    def new_message(self, content, state):
        message = Message(content, state)
        self.inbox.append(message)
        
    def display_inbox(self):
        inbox_str = "-"*50
        inbox_str += "\n"
        inbox_str += f"Inbox for Contact {self.alias} \n"
        
        inbox_str += "\n"
        inbox_str = "-"*50
        inbox_str += "\n"
        inbox_str += f"Connection ID : {self.connection_id} \nAgent Label : {self.agent_label} \nTotal Messages : {len(self.inbox)}"
        inbox_str += "\n\n"
        inbox_str += "-"*50
        for message in self.inbox:
            inbox_str += "\n"
            inbox_str += message.to_string()
        return inbox_str

In [6]:
from datetime import datetime

class Message:

    def __init__(self, content, state):
        self.content = content
        self.state = state
        self.time = datetime.now()
        
    def to_string(self):
        msg_str = self.time.strftime("%c")
        n = 25
        words = self.content.split()
        
        msg_chunks = [self.content[i:i+n] for i in range(0, len(self.content), n)]
        line_start = ""
        if self.state == "sent":
            line_start += (" "*25)
        current_line = ""
        for word in words:
            current_line += (word + " ")
            if len(current_line) > 25:
                msg_str += "\n"
                msg_str += (line_start + current_line)
                current_line = ""
        if not current_line == "":
            msg_str += "\n"
            msg_str += (line_start + current_line)
        msg_str += "\n"
        msg_str += ("-"*50)
        return msg_str

In [7]:
message = Message("Some really quite long content how long is really long though I mean come on", "received")
print(message.to_string())

Tue Jun  1 15:05:22 2021
Some really quite long content 
how long is really long though 
I mean come on 
--------------------------------------------------


In [8]:
import json

class MessagingService:
    
    def __init__(self, agent_controller):
        self.agent_controller = agent_controller
        self.contacts: list[Contact] = []
        
        listeners = [
            {
                "handler": self._messages_handler,
                "topic": "basicmessages"
            },
            {
                "handler": self._connections_handler,
                "topic": "connections"
            }
        ]
        
        self.agent_controller.register_listeners(listeners)
        
    
    def _messages_handler(self, payload):
        connection_id = payload["connection_id"]
        contact = self.find_contact_by_id(connection_id)
        if contact:
            print(f"Message received from connection {connection_id} with alias {contact.alias}")
            content = payload["content"]
            state = "received"
            
           
            contact.new_message(content, state)
    
    # Receive connection messages
    def _connections_handler(self, payload):
        state = payload['state']
        connection_id = payload["connection_id"]
        their_role = payload["their_role"]
        routing_state = payload["routing_state"]
        their_label = payload["their_label"]

#         if state == "invitation":
#             # Your business logic
#             print("invitation")
#         elif state == "request":
#             # Your business logic
#             print("request")

#         elif state == "response":
#             # Your business logic
#             print("response")
        if state == "active":
            contact = self.find_contact_by_id(connection_id)
            if contact:
                contact.is_active.set_result(True)
                contact.agent_label = their_label
                print(colored("Contact with ID: {0} and alias {1} is now active.".format(connection_id, contact.alias), "green", attrs=["bold"]))
            else:
                print(f"No contact for active connection with ID {connection_id}")
                
            # Your business logic
            
        
    def find_contact_by_id(self, connection_id):
        contact_exists = False
        for contact in self.contacts:
            if contact.connection_id == connection_id:
                return contact
        return None
    
    def display_inbox_for_contact(self, connection_id):
        contact = self.find_contact_by_id(connection_id)
        if not contact:
            print(f"No contact saved for connection with id {connection_id}")
            return
        print(contact.display_inbox())
    
    def view_inbox(self):
        inbox_view = "-"* 50
        inbox_view += "\n"
        inbox_view += f"{len(self.contacts)} Contacts"
        inbox_view += "\n"
        inbox_view = "-"* 50
        inbox_view += "\n"
        for contact in self.contacts:
            inbox_view += f"Contact {contact.alias} with ID {contact.id} \n"
            inbox_view += f"Messages {len(contact.messages)} \n"
            inbox_view = "-"* 50
            inbox_view += "\n"
        print(inbox_view)
        
        
    
    
    def send_message(self, connection_id, content):
        contact = self.find_contact_by_id(connection_id)
        if contact:
            if contact.is_active:
                state = "sent"
                loop = asyncio.get_event_loop()
                loop.run_until_complete(self.agent_controller.messaging.send_message(connection_id, content))
                contact.new_message(content, state)
            else:
                print(f"Contact {contact.alias} is not yet an active connection")
        else: 
            print(f"No contact saved for connection id {connection_id}")
        
    def add_contact(self, connection_id, alias):
        contact = Contact(connection_id, alias)
        print(f"Contact with ID {connection_id} and alias {alias} added")
        self.contacts.append(contact)
    
    def new_contact_invite(self, alias):
        auto_accept = True
        # Use public DID?
        public = "false"
        # Should this invitation be usable by multiple invitees?
        multi_use = "false"
        loop = asyncio.get_event_loop()
        response = loop.run_until_complete(agent_controller.connections.create_invitation(alias, auto_accept, public, multi_use))
        invitation = response["invitation"]
        connection_id = response["connection_id"]
        self.add_contact(connection_id, alias)
        return json.dumps(invitation)
        
    
    def accept_contact_invitation(self, invitation, alias, label = None):
        loop = asyncio.get_event_loop()
        auto_accept="false"
        response = loop.run_until_complete(agent_controller.connections.receive_invitation(invitation,alias, auto_accept))
        connection_id = response["connection_id"]
        self.add_contact(connection_id, alias)

        
        # Endpoint you expect to recieve messages at
        my_endpoint = None

        accept_response = loop.run_until_complete(agent_controller.connections.accept_invitation(connection_id, label, my_endpoint))
        return connection_id
            

In [9]:
messaging_service = MessagingService(agent_controller)

In [10]:
invitation = {'@type': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/connections/1.0/invitation', '@id': '1e6f60e5-071c-41c9-977b-ca27007c278d', 'serviceEndpoint': 'https://0025438b5333.ngrok.io', 'label': 'Hyperledger Global Forum', 'recipientKeys': ['4bTvq7oN64hYqUwMedQ3b5aHUo7WjKfTuMZnqA2bAfzw']}

In [11]:
new_contact_id = messaging_service.accept_contact_invitation(invitation, "HLGF")

----------------------------------------------------------
Connection Webhook Event Received
Connection ID :  f489913b-2d7d-45e0-a924-8d7b9738d579
State :  invitation
Routing State :  none
Their Role :  inviter
----------------------------------------------------------
invitation
Contact with ID f489913b-2d7d-45e0-a924-8d7b9738d579 and alias HLGF added
----------------------------------------------------------
Connection Webhook Event Received
Connection ID :  f489913b-2d7d-45e0-a924-8d7b9738d579
State :  request
Routing State :  none
Their Role :  inviter
----------------------------------------------------------
request
----------------------------------------------------------
Connection Webhook Event Received
Connection ID :  f489913b-2d7d-45e0-a924-8d7b9738d579
State :  response
Routing State :  none
Their Role :  inviter
----------------------------------------------------------
response
----------------------------------------------------------
Connection Webhook Event Received


In [12]:
messaging_service.send_message(new_contact_id, "Some second super super super great test message")

Message received from connection f489913b-2d7d-45e0-a924-8d7b9738d579 with alias HLGF


In [13]:
messaging_service.display_inbox_for_contact(new_contact_id)

--------------------------------------------------

Inbox for Contact HLGF 
 Connection ID : f489913b-2d7d-45e0-a924-8d7b9738d579 
 Agent Label : Hyperledger Global Forum 
 Total Messages : 2

--------------------------------------------------
Tue Jun  1 15:05:48 2021
                         Some second super super super 
                         great test message 
--------------------------------------------------
Tue Jun  1 15:05:48 2021
Hyperledger Global Forum received 
your message 
--------------------------------------------------


## Terminate Controller

Whenever you have finished with this notebook, be sure to terminate the controller. This is especially important if your business logic runs across multiple notebooks.

In [ ]:
await agent_controller.terminate()